In [1]:
import zipfile
import os


zip_path = '/content/landscape_pictures.zip'
extract_to = 'unzipped_landscape_pictures'


os.makedirs(extract_to, exist_ok=True)


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Files extracted to:", extract_to)

Files extracted to: unzipped_landscape_pictures


In [2]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])



In [3]:
import torch
import torch.nn as nn
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms, utils
from tqdm import tqdm
import wandb


wandb.login(key="your_wandb_api_key_here")  # Replace with your actual WandB API key
wandb.init(project="conv-autoencoder-256", name="deep-ae-256x256", reinit=True)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 777bhavya (777bhavya-dwarkadas-j-sanghvi-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


In [5]:
class ImageFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [os.path.join(folder_path, f)
                            for f in os.listdir(folder_path)
                            if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img = Image.open(self.image_files[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img 


img_folder = "/content/unzipped_landscape_pictures"

# Dataset and DataLoader
dataset = ImageFolderDataset(img_folder, transform=transform)
# dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Check one batch



In [6]:



val_ratio = 0.1
val_size = int(len(dataset) * val_ratio)
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)


In [7]:
import torch
import torch.nn as nn

class DeepConvAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()

        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),   # 256 → 128
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), # 128 → 64
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),# 64 → 32
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),# 32 → 16
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1), # 16 → 32
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1), # 32 → 64
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 64 → 128
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),    # 128 → 256
            nn.Sigmoid()  # pixel values between 0 and 1
        )

    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out


In [8]:



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepConvAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [15]:
EPOCHS = 20
def log_reconstructions(images, reconstructions, label):
    img_grid = make_grid(torch.cat([images[:8], reconstructions[:8]], dim=0), nrow=8, normalize=True)
    wandb.log({label: [wandb.Image(img_grid)]})

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{EPOCHS}]")

    for images in loop:
        images = images.to(device)

        recon = model(images)
        loss = criterion(recon, images)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        loop.set_postfix(train_loss=loss.item())

    train_loss /= len(train_loader)


    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images  in val_loader:
            images = images.to(device)
            recon = model(images)
            loss = criterion(recon, images)
            val_loss += loss.item()
    val_loss /= len(val_loader)


    wandb.log({
        "train_loss": train_loss,
        "val_loss": val_loss,
        "epoch": epoch + 1
    })


    with torch.no_grad():
        val_imgs = next(iter(val_loader))
        val_imgs = val_imgs.to(device)
        recon = model(val_imgs)
        log_reconstructions(val_imgs.cpu(), recon.cpu(), label="Reconstruction")


Epoch [20/20]: 100%|██████████| 31/31 [01:20<00:00,  2.59s/it, train_loss=0.176]
